In [10]:
import pandas as pd 
import os 
from pathlib import Path  
import utility as ut # see scripts/utilities.py 
import matplotlib.pyplot as plt
import numpy as np
import openpyxl # use read_excel from pd


In [11]:
# rootfolder = Path.cwd() 
rootfolder = Path("/Users/bingli/Documents/GitHub/fMRI-AHDH") # This could be changed to the root dir in repo 
traindata = os.path.join(rootfolder, "data", "TRAIN")
file_lst = ut.extract_file_path(traindata)

/Users/bingli/Documents/GitHub/fMRI-AHDH/data/TRAIN/TRAIN_QUANTITATIVE_METADATA.xlsx
/Users/bingli/Documents/GitHub/fMRI-AHDH/data/TRAIN/TRAINING_SOLUTIONS.xlsx
/Users/bingli/Documents/GitHub/fMRI-AHDH/data/TRAIN/TRAIN_CATEGORICAL_METADATA.xlsx
/Users/bingli/Documents/GitHub/fMRI-AHDH/data/TRAIN/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv


In [15]:
train_quant = pd.read_excel(file_lst[0])
train_cate = pd.read_excel(file_lst[2])
train_outcome = pd.read_excel(file_lst[1]) 
train_fmri = pd.read_csv(file_lst[3])

In [22]:
train_cate.head()
train_cate_encoded = pd.get_dummies(train_cate, columns= train_cate.columns[1:], drop_first=True)
train_cate_encoded.head()

,participant_id,Basic_Demos_Enroll_Year_2016,Basic_Demos_Enroll_Year_2017,Basic_Demos_Enroll_Year_2018,Basic_Demos_Enroll_Year_2019,Basic_Demos_Enroll_Year_2020,Basic_Demos_Study_Site_2,Basic_Demos_Study_Site_3,Basic_Demos_Study_Site_4,PreInt_Demos_Fam_Child_Ethnicity_1.0,...,Barratt_Barratt_P2_Edu_21,Barratt_Barratt_P2_Occ_5,Barratt_Barratt_P2_Occ_10,Barratt_Barratt_P2_Occ_15,Barratt_Barratt_P2_Occ_20,Barratt_Barratt_P2_Occ_25,Barratt_Barratt_P2_Occ_30,Barratt_Barratt_P2_Occ_35,Barratt_Barratt_P2_Occ_40,Barratt_Barratt_P2_Occ_45
0,UmrK0vMLopoR,True,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
1,CPaeQkhcjg7d,False,False,False,True,False,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False
2,Nb4EetVPm3gs,True,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,p4vPhVu91o4b,False,False,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,M09PXs7arQ5E,False,False,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [4]:
# Checking missing values 
for col in train_cate.columns: 
    ut.missing_percentage(train_cate, col)
# Interesting to see only one column "PreInt_Demos_Fam_Child_Ethnicity" has a relatively high (1%) missing percentage

for col in train_quant.columns:
    ut.missing_percentage(train_quant, col)
# Interesting to see only one column "MRI_Track_Age_at_Scan" has a very high (29.7%) missing percentage 

for col in train_fmri.columns: # Nothing is missing 
    ut.missing_percentage(train_fmri, col) 

col PreInt_Demos_Fam_Child_Ethnicity has 0.9068425391591096 % missing.
col MRI_Track_Age_at_Scan has 29.67848309975268 % missing.


In [5]:
# MCAR test for MRI_Track_Age_at_Scan column 
# Goal: 1) if data is missing completely at random (MCAR); 
# 2) Missing at random (MAR); 
# 3) Missing NOT at random (MNAR) 

# see if the missing / presence of columns with missing are correlated with any columns in the dataset
# Create an indicator variable for missing values
train_quant["MRI_missing"] = np.where(train_quant["MRI_Track_Age_at_Scan"].isnull(), 1, 0)
train_cate["child_eth_missing"] = np.where(train_cate["PreInt_Demos_Fam_Child_Ethnicity"].isnull(), 1, 0)

pa_id = list(set(train_cate.columns) & set(train_quant.columns))[0]
train_demo = train_cate.merge(train_quant, on=pa_id) 

numerical_columns = train_demo.select_dtypes(include=['number']).columns.tolist()
categorical_columns = train_demo.select_dtypes(include=['object', 'category']).columns.tolist()
print(len(train_demo.columns))
print(len(numerical_columns) + len(categorical_columns)) # check if all included 
numerical_columns = [col for col in numerical_columns if col not in ["MRI_missing", "child_eth_missing"]]

30
30


In [8]:
ut.chi2_with_one_columns(categorical_columns, "MRI_missing", train_demo)
# It doesn't seem to have any column has a correlation with missing values in "MRI_Track_Age_at_Scan"

Chi-Square Test between participant_id and MRI_missing:
Chi-Square Stat: 1213.00, P-value: 0.48650


In [ ]:
ut.chi2_with_one_columns